# Izračun razlike višine in azimuta nebesnega telesa (SRP - Sight Reduction Procedure)

To je metoda po Mark De Saint Hailare

In [ ]:
import os, sys

# add custom modules and astro data path 
pp = '../nav_tools/'
sys.path.append(pp)

# needed by basemap
os.environ['PROJ_LIB'] = '/opt/miniconda3/envs/astrolab/share/proj'

In [ ]:
import math as mat
import numpy as np
import matplotlib.pyplot as mpl
mpl.rcParams['text.usetex'] = True
mpl.rcParams.update({'font.size': 7})

import celestialdata as cdata
import navigationalstars as ns
import navtools as nt